In [1]:
#Parse sample.sam to only include lines with Chr
#from those lines only keep the Chr name and start position
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/sample.sam") as sam:
    out = open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Sample.txt", 'w')
    
    for line in sam:
        col = line.split('\t')
        
        if "*" not in col[2]:
            out.write(col[2]+'\t'+col[3]+'\n')
            
out.close()

In [2]:
#Parse arabidopsis.gtf to only include lines with exon in it
#from those lines only keep Chr name, start pos, end pos and gene name
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Arabidopsis.gtf") as ara:
    out = open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Arabidopsis.txt", 'w')
    
    for line in ara:
        col = line.split('\t')
        gene = col[8].split('"')
        
        if "exon" in col[2]:
            out.write(col[0]+'\t'+col[3]+'\t'+col[4]+'\t'+gene[5]+'\n')
            
out.close()

In [3]:
#go through the parsed sample file and create a dictionary to match
#chromosomes(keys) with start points(values) from parsed sample file
match={}
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Sample.txt") as sam:
    
    for line in sam:
        col = line.strip('\n').split('\t')
        
        if col[0] in match.keys():
            match[col[0]].append(int(col[1]))
        else:
            match[col[0]] = [int(col[1])]
            
#after populating the list, replace chloroplast and mitochondria with 
#ChrC and ChrM respecitvely to match the arabidopsis notation           
match['ChrM'] = match['mitochondria']
del match['mitochondria']
match['ChrC'] = match['chloroplast']
del match['chloroplast']

In [4]:
#populate keys in dict ping from parsed atabidopsis file
#and set all values to 0
ping={}
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Arabidopsis.txt") as par:
    
    for line in par:
        col = line.strip('\n').split('\t')
        
        if col[0] in match.keys():
            ping[col[3]]=0

In [5]:
#UNSORTED RUN
#go through each line in parsed arabidopsis and for each chr in
#match dict that mathces the chr in arabidopsis compare the start point from
#match and see if it falls in the arabidopsis range
#if it matches uptick the counter for that chr in ping dict
#then pop value out as long as a match was made
import time
start=time.process_time()
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Arabidopsis.txt") as par:
    
    for line in par:
        col = line.strip('\n').split('\t')
        
        for hit in match[col[0]]:
            if (hit>=int(col[1]) and hit<=int(col[2])):
                if col[3] in ping.keys():
                    ping[col[3]] +=1
                    
                match[col[0]].pop(0)
                break

print("Time to complete UNSORTED run = "+str(time.process_time() - start)+" seconds")
import pandas as pd
pingPD=pd.DataFrame(ping.items())
pingPD.head()

Time to complete UNSORTED run = 5326.421875 seconds


,0,1
0,AT1G01010,1
1,AT1G01020,2
2,AT1G01030,0
3,AT1G01040,10
4,AT1G01046,0


In [6]:
pingPD.tail()

,0,1
33605,ATMG01370,0
33606,ATMG01380,1
33607,ATMG01390,1
33608,ATMG01400,0
33609,ATMG01410,0


In [7]:
#Sorting function using merge sort
def MS(grp):
    if len(grp)>1:
        #assign value to the middle of the list
        mid = len(grp)//2
        #split the list in 2, L being the left and R being the right
        L = grp[:mid]
        R = grp[mid:]
        #cal the function to split the Left and Right side indefinetly until only 1 element remains in each split
        MS(L)
        MS(R)
        #creat values to act as trackers
        x = y = z = 0
        #use temporary values to hold and sort value while resorting
        while x < len(L) and y < len(R):
            if L[x] < R[y]:
                grp[z] = L[x]
                x += 1
            else:
                grp[z] = R[y]
                y += 1
            z += 1
        #Make sure there are no values left in the splits
        while x < len(L):
            grp[z] = L[x]
            x += 1
            z += 1
        while y < len(R):
            grp[z] = R[y]
            y += 1
            z += 1

In [8]:
#sort the starting points for each chromosome in our match dict
for chro in match:
    MS(match.get(chro))

In [9]:
#reset the keys in dict ping and set all values to 0 again
ping={}
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Arabidopsis.txt") as par:
    
    for line in par:
        col = line.strip('\n').split('\t')
        
        if col[0] in match.keys():
            ping[col[3]]=0

In [10]:
#SORTED RUN
import time
start=time.process_time()
with open("/Users/Midaroh/Desktop/School/Algorithims and Data Structures/Assignments/SandBox/Parsed_Arabidopsis.txt") as par:
    
    for line in par:
        col = line.strip('\n').split('\t')
        
        while (len(match[col[0]])>0):
            hit = match[col[0]].pop(0)
            
            if (hit>=int(col[1]) and hit<=int(col[2])):
                ping[col[3]] +=1
            
            elif hit > int(col[2]):
                match[col[0]].insert(0,hit)
                break

print("Time to complete SORTED run = "+str(time.process_time() - start)+" seconds")
import pandas as pd
pingPD=pd.DataFrame(ping.items())
pingPD.head()

Time to complete SORTED run = 120.65625 seconds


,0,1
0,AT1G01010,1
1,AT1G01020,0
2,AT1G01030,0
3,AT1G01040,2
4,AT1G01046,0


In [11]:
pingPD.tail()

,0,1
33605,ATMG01370,0
33606,ATMG01380,25
33607,ATMG01390,3615
33608,ATMG01400,0
33609,ATMG01410,0


Amir Elbahnasawi (ae1142)